In [2]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [7]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [15]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")